In [ ]:
import requests
import pymongo
import json
import time

In [ ]:
#MongoDB CLient connection and create local DB and collection
client = pymongo.MongoClient("mongodb://localhost:27017/")
politicsData = client["redditDB"]
localCollection = politicsData["postCollection"]
commentCollection = politicsData["commentCollection"]

#Get login information from text file
logininfo = []
with open('logininfo.txt', 'r') as f:
    templogininfo = f.readlines()
for line in templogininfo:
    logininfo = logininfo + [line.strip()]
CLIENT_ID = logininfo[3]
SECRET_KEY = logininfo[4]


In [ ]:
#Reddit api authentication and connection
auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_KEY)
headers = {}
data =  {
    'grant_type': 'password',
    'username': logininfo[0],
    'password': logininfo[1]
}
reddit_url = 'https://www.reddit.com/'
api_url = 'https://oauth.reddit.com'

res = requests.post(reddit_url+'api/v1/access_token', auth = auth, data = data,headers = {'User-Agent': logininfo[2]})
print(res.status_code)
if res.status_code == 200:
    resData = res.json()
    TOKEN = 'bearer '+ resData['access_token']
    headers['User-Agent'] = logininfo[2]
    headers['Authorization'] = TOKEN
    requests.get(api_url+'/api/v1/me', headers = headers)

In [ ]:
#Get subreddits and sorting topics from text files
srNameList = []
with open('subreddit.txt', 'r') as fsub:
    tempsubinfo = fsub.readlines()
for line in tempsubinfo:
    srNameList = srNameList + [line.strip()]

print(srNameList)
    
srSortList = []
with open('sorting.txt', 'r') as fsrt:
    tempsortinfo = fsrt.readlines()
for line in tempsortinfo:
    srSortList = srSortList + [line.strip()]

print(srSortList)

In [ ]:
#Function to get comments' data
def getCommentData(srName,srSort,postidTemp,permalinkTemp):
    comRes = requests.get(api_url+permalinkTemp,headers = headers,params = {'limit':'100','depth':1})
    if comRes.status_code==200:
        comTemp = comRes.json()
        comJson = comTemp[1]['data']['children']
        for eachCom in comJson:
            if eachCom['kind'] == 't1':
                com_dictionary = []
                if (commentCollection.count_documents({"comment_id":eachCom['data']['id']},limit=1) == 0):
                    com_dictionary = {
                        'subreddit': eachCom['data']['subreddit_name_prefixed'],
                        'postid': postidTemp,
                        'comment_id': eachCom['data']['id'],
                        'comment_text': eachCom['data']['body'],
                        'created_utc': eachCom['data']['created_utc']}
                    commentCollection.insert_one(com_dictionary)
                com_dictionary = []
        time.sleep(2)

In [ ]:
#Function to get posts' data
def getPostData():
    oldDocumentCount = 0
    newDocumentCount = 0
    iterationNo = 1
    while True:
        for srName in srNameList:
            print(srName)
            for srSort in srSortList:
                print(srSort)
                firstid = 0
                po_id = {}
                updatedFlag = 1
                if (firstid == 0):
                    politicsData = requests.get(api_url+'/r/'+srName+'/'+srSort,headers = headers , params={'limit':'1'})
                    for post in politicsData.json()['data']['children']:
                        po_id = politicsData.json()['kind']+'_'+post['data']['id']
                        
                    firstid = 1
                if (firstid == 1):
                    while (updatedFlag!=0):
                        oldDocumentCount = newDocumentCount
                        politicsData = requests.get(api_url+'/r/'+srName+'/'+srSort,headers = headers , params={'limit':'100','after':{po_id}})
                        politicsData.json()
                        po_id = politicsData.json()['data']['after']
                        print(po_id)
                        for post in politicsData.json()['data']['children']:
                            new_dictionary = []
                            postidTemp = post['data']['id']
                            permalinkTemp = post['data']['permalink']
                            if (localCollection.count_documents({"postid":post['data']['id']},limit=1) == 0):                        
                                new_dictionary = {
                                    'subreddit': post['data']['subreddit_name_prefixed'],
                                    'postid': post['data']['id'],
                                    'created_utc': post['data']['created_utc'],
                                    'title':post['data']['title'],
                                    'selftext':post['data']['selftext'],
                                    'permalink':post['data']['permalink'],
                                    'num_comments':post['data']['num_comments'],
                                    'upvotes':post['data']['ups'],
                                    'upvote_ratio':post['data']['upvote_ratio'],
                                    'url': post['data']['url']}
                                
                                localCollection.insert_one(new_dictionary)
                                newDocumentCount = newDocumentCount + 1
                            else:
                                getCommentData(srName,srSort,postidTemp,permalinkTemp)
                            new_dictionary = []
                        time.sleep(2)
                        if(newDocumentCount - oldDocumentCount != 0):
                            updatedFlag = 1
                            iterationNo = iterationNo + 1
                        else:
                            updatedFlag = 0

In [ ]:
def main(): 
    try:
        getPostData()
    except Exception as exception:
        file = open('error.txt', 'w+')
        file.write('Error: %s' %exception)
        file.close()

In [ ]:
if __name__ == "__main__":
    main()